# Access a Database with Python - Iris Dataset


Iris flower data set is a popular dataset in machine learning. The data set consists of 50 samples from each of three species of Iris: Iris setosa, Iris virginica, and Iris versicolor. 
Four features were measured from each sample: the length and the width of the sepals and petals, in centimetres. 

![Three species of Iris](iris_types.jpg)

First, let's check if the sqlite database is available and display an error message if the file is not available:

In [245]:
import os
dr = '../iris-species'
sql_fn = "database.sqlite" # containing the main data set
sql_cc_fn = 'iris_cc.sqlite' # containing the diploid chromosome counts for each species of iris
data_iris_folder_content = os.listdir(dr)

In [246]:
data_iris_folder_content

['.DS_Store', 'database.sqlite', 'Iris.csv', 'iris_cc.csv', 'iris_cc.sqlite']

In [247]:
error_message = "Error: sqlite file not available"
assert sql_fn in data_iris_folder_content, error_message

## Access the Database with the sqlite3 Package

We can use the `sqlite3` package from the Python standard library to connect to the `sqlite` database:

In [10]:
import sqlite3

If you need to connect to another database, you would use another package instead of `sqlite3`, for example:

* [MySQL Connector](https://dev.mysql.com/doc/connector-python/en/) for MySQL
* [Psycopg](http://initd.org/psycopg/docs/install.html) for PostgreSQL
* [pymssql](http://pymssql.org/en/stable/) for Microsoft MS SQL

In [258]:
# connect to the database
conn = sqlite3.connect(os.path.join(dr, sql_fn))

In [259]:
os.path.join(dr, sql_fn)

'../iris-species/database.sqlite'

In [260]:
# A `sqlite3.Cursor` object is our interface to the database
cursor = conn.cursor()

In [261]:
type(cursor)

sqlite3.Cursor

We can get a list of all the tables saved into the database, this is done by reading the column `name` from the `sqlite_master` metadata table with.

In [262]:
cursor.execute("SELECT name FROM sqlite_master").fetchall()

[(u'Iris',), (u'ChromCount',), (u'sqlite_autoindex_ChromCount_1',)]

Then we can execute standard `SQL` query on the database. 

Let's take a look for example at the first 5 rows in the Iris table:

In [19]:
sample_data = cursor.execute("SELECT * FROM Iris LIMIT 5").fetchall()

In [21]:
sample_data

[(1, 5.1, 3.5, 1.4, 0.2, u'Iris-setosa'),
 (2, 4.9, 3, 1.4, 0.2, u'Iris-setosa'),
 (3, 4.7, 3.2, 1.3, 0.2, u'Iris-setosa'),
 (4, 4.6, 3.1, 1.5, 0.2, u'Iris-setosa'),
 (5, 5, 3.6, 1.4, 0.2, u'Iris-setosa')]

It is evident that the interface provided by `sqlite3` is low-level, for data exploration purposes we would like to directly import data into a more user friendly library like `pandas`.

## Import data from a database to `pandas`

In [23]:
import pandas as pd

In [45]:
iris_df = pd.read_sql_query("SELECT * FROM Iris", conn)

`pandas.read_sql_query` takes a `SQL` query and a connection object and imports the data into a `DataFrame`, also keeping the same data types of the database columns. `pandas` provides a lot of the same functionality of `SQL` with a more user-friendly interface.

In [46]:
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [47]:
iris_df.shape

(150, 6)

In [48]:
iris_df.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

However, `sqlite3` is extremely useful for downselecting data **before** importing them in `pandas`. For example you might have 1 TB of data in a table stored in a database on a server machine. You are interested in working on a subset of the data based on some criterion, unfortunately it would be impossible to first load data into `pandas` and then filter them, therefore we should tell the database to perform the filtering and just load into `pandas` the downsized dataset.

In [49]:
iris_setosa_df = pd.read_sql_query("SELECT * FROM Iris WHERE Species == 'Iris-setosa'", conn)

In [50]:
iris_setosa_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [51]:
iris_setosa_df.shape

(50, 6)

## Compare `SQLite` and `pandas` results 

We practice running various commands using both SQLite and pandas libraries, i.e. either passing SQL queries to our database or working with our pandas data frame, respectively.  

### Select all records from specific fields

In [75]:
iris_l = cursor.execute('SELECT Species FROM Iris').fetchall()
print iris_l[:5]
len(iris_l)

[(u'Iris-setosa',), (u'Iris-setosa',), (u'Iris-setosa',), (u'Iris-setosa',), (u'Iris-setosa',)]


150

In [69]:
print iris_df.Species[:5]
len(iris_df.Species)

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Species, dtype: object


150

Select all records from multiple fields:

In [76]:
iris_l = cursor.execute('SELECT Species, PetalWidthCm FROM Iris').fetchall()
print iris_l[:5]

[(u'Iris-setosa', 0.2), (u'Iris-setosa', 0.2), (u'Iris-setosa', 0.2), (u'Iris-setosa', 0.2), (u'Iris-setosa', 0.2)]


In [107]:
iris_df[['Species', 'PetalWidthCm']].head()

,Species,PetalWidthCm
0,Iris-setosa,0.2
1,Iris-setosa,0.2
2,Iris-setosa,0.2
3,Iris-setosa,0.2
4,Iris-setosa,0.2


### Select unique occurrences from specified field

In [78]:
iris_l = cursor.execute('SELECT DISTINCT Species FROM Iris').fetchall()
iris_l

[(u'Iris-setosa',), (u'Iris-versicolor',), (u'Iris-virginica',)]

In [80]:
iris_df.Species.unique()

array([u'Iris-setosa', u'Iris-versicolor', u'Iris-virginica'], dtype=object)

Select unique occurrences from multiple fields:

In [83]:
iris_l = cursor.execute('SELECT DISTINCT Species, PetalWidthCm FROM Iris').fetchall()
iris_l[:7]

[(u'Iris-setosa', 0.2),
 (u'Iris-setosa', 0.4),
 (u'Iris-setosa', 0.3),
 (u'Iris-setosa', 0.1),
 (u'Iris-setosa', 0.5),
 (u'Iris-setosa', 0.6),
 (u'Iris-versicolor', 1.4)]

In [88]:
iris_df[['Species', 'PetalWidthCm']].drop_duplicates().head(7)

,Species,PetalWidthCm
0,Iris-setosa,0.2
5,Iris-setosa,0.4
6,Iris-setosa,0.3
9,Iris-setosa,0.1
23,Iris-setosa,0.5
43,Iris-setosa,0.6
50,Iris-versicolor,1.4


### Filter records

In [161]:
iris_l = cursor.execute("SELECT * FROM Iris WHERE Species='Iris-setosa'").fetchall()
print len(iris_l)
iris_l[:5]

50


[(1, 5.1, 3.5, 1.4, 0.2, u'Iris-setosa'),
 (2, 4.9, 3, 1.4, 0.2, u'Iris-setosa'),
 (3, 4.7, 3.2, 1.3, 0.2, u'Iris-setosa'),
 (4, 4.6, 3.1, 1.5, 0.2, u'Iris-setosa'),
 (5, 5, 3.6, 1.4, 0.2, u'Iris-setosa')]

In [142]:
iris_df.loc[iris_df.Species == 'Iris-setosa'].head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


#### `Between` and `In` examples:

In [150]:
iris_l = cursor.execute("SELECT * FROM Iris WHERE PetalWidthCm BETWEEN 0.2 AND 0.5").fetchall()
print len(iris_l)
iris_l[:5]

43


[(1, 5.1, 3.5, 1.4, 0.2, u'Iris-setosa'),
 (2, 4.9, 3, 1.4, 0.2, u'Iris-setosa'),
 (3, 4.7, 3.2, 1.3, 0.2, u'Iris-setosa'),
 (4, 4.6, 3.1, 1.5, 0.2, u'Iris-setosa'),
 (5, 5, 3.6, 1.4, 0.2, u'Iris-setosa')]

In [153]:
d = iris_df.loc[iris_df.PetalWidthCm.between(.2,.5)]
print d.shape
d.head()

(43, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [154]:
iris_l = cursor.execute("SELECT * FROM Iris WHERE PetalWidthCm IN (0.2, 0.5)").fetchall()
print len(iris_l)
iris_l[:5]

29


[(1, 5.1, 3.5, 1.4, 0.2, u'Iris-setosa'),
 (2, 4.9, 3, 1.4, 0.2, u'Iris-setosa'),
 (3, 4.7, 3.2, 1.3, 0.2, u'Iris-setosa'),
 (4, 4.6, 3.1, 1.5, 0.2, u'Iris-setosa'),
 (5, 5, 3.6, 1.4, 0.2, u'Iris-setosa')]

In [157]:
d = iris_df.loc[iris_df.PetalWidthCm.isin([.2,.5])]
print d.shape
d.head()

(29, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Filter records with pattern matching

In [162]:
iris_l = cursor.execute("SELECT * FROM Iris WHERE Species LIKE 'Iris-s%'").fetchall()
print len(iris_l)
iris_l[:5]

50


[(1, 5.1, 3.5, 1.4, 0.2, u'Iris-setosa'),
 (2, 4.9, 3, 1.4, 0.2, u'Iris-setosa'),
 (3, 4.7, 3.2, 1.3, 0.2, u'Iris-setosa'),
 (4, 4.6, 3.1, 1.5, 0.2, u'Iris-setosa'),
 (5, 5, 3.6, 1.4, 0.2, u'Iris-setosa')]

In [164]:
d = iris_df.loc[iris_df['Species'].str.match('Iris-s')]
print d.shape
d.head()

(50, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Group by aggregation

In [173]:
iris_l = cursor.execute('SELECT Species, COUNT(*) FROM Iris GROUP BY Species').fetchall()
iris_l

[(u'Iris-setosa', 50), (u'Iris-versicolor', 50), (u'Iris-virginica', 50)]

In [197]:
ser = iris_df.groupby('Species')['PetalWidthCm']

In [198]:
ser.describe()

,count,mean,std,min,25%,50%,75%,max
Species,,,,,,,,
Iris-setosa,50.0,0.244,0.107210,0.1,0.2,0.2,0.3,0.6
Iris-versicolor,50.0,1.326,0.197753,1.0,1.2,1.3,1.5,1.8
Iris-virginica,50.0,2.026,0.274650,1.4,1.8,2.0,2.3,2.5


In [205]:
ser.mean()

Species
Iris-setosa        0.244
Iris-versicolor    1.326
Iris-virginica     2.026
Name: PetalWidthCm, dtype: float64

In [201]:
d = iris_df.groupby(['Species', 'PetalWidthCm'])['PetalLengthCm'].count()
d.head(10)

Species          PetalWidthCm
Iris-setosa      0.1              6
                 0.2             28
                 0.3              7
                 0.4              7
                 0.5              1
                 0.6              1
Iris-versicolor  1.0              7
                 1.1              3
                 1.2              5
                 1.3             13
Name: PetalLengthCm, dtype: int64

### Multiple Statistics per Group

In [213]:
# Applying a single function to columns in groups
iris_df.groupby('Species').agg({'PetalWidthCm': ['mean'],
                                'PetalLengthCm': ['min'],
                                'SepalLengthCm': ['max']})

,PetalWidthCm,PetalLengthCm,SepalLengthCm
,mean,min,max
Species,,,
Iris-setosa,0.244,1.0,5.8
Iris-versicolor,1.326,3.0,7.0
Iris-virginica,2.026,4.5,7.9


In [215]:
# Applying multiple functions to columns in groups
iris_df.groupby('Species').agg({'PetalWidthCm': ['min', 'max', 'mean'],
                                'PetalLengthCm': ['min', 'max', 'mean'],
                                'SepalLengthCm': 'first'})

PetalWidthCm             PetalLengthCm              \
                         min  max   mean           min  max   mean   
Species                                                              
Iris-setosa              0.1  0.6  0.244           1.0  1.9  1.464   
Iris-versicolor          1.0  1.8  1.326           3.0  5.1  4.260   
Iris-virginica           1.4  2.5  2.026           4.5  6.9  5.552   

                SepalLengthCm  
                        first  
Species                        
Iris-setosa               5.1  
Iris-versicolor           7.0  
Iris-virginica            6.3

### Frequency

In [216]:
cursor.execute("SELECT Species, COUNT(*) FROM Iris GROUP BY Species").fetchall()

[(u'Iris-setosa', 50), (u'Iris-versicolor', 50), (u'Iris-virginica', 50)]

In [39]:
iris_data.Species.value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

### Order By to sort data

In [228]:
iris_l = cursor.execute("SELECT PetalWidthCm FROM Iris ORDER BY PetalWidthCm DESC").fetchall()
iris_l[:10]

[(2.5,),
 (2.5,),
 (2.5,),
 (2.4,),
 (2.4,),
 (2.4,),
 (2.3,),
 (2.3,),
 (2.3,),
 (2.3,)]

In [229]:
iris_data.PetalWidthCm.sort_values(ascending=False).head(10)

109    2.5
144    2.5
100    2.5
114    2.4
140    2.4
136    2.4
135    2.3
118    2.3
145    2.3
143    2.3
Name: PetalWidthCm, dtype: float64

### JOIN to merge shared records across two tables

Read the second database containing the diploid chromosome counts for each species of iris:

In [343]:
iris_cc_df = pd.read_sql_query('SELECT * FROM ChromCount', conn)

In [344]:
iris_cc_df.head()

,Name,DipChromCount
0,Iris-setosa,38
1,Iris-versicolor,108
2,Iris-virginica,70


In [265]:
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [289]:
iris_l = cursor.execute('SELECT Iris.Species, ChromCount.DipChromCount '\
                        'FROM Iris INNER JOIN ChromCount '\
                        'ON Iris.Species=ChromCount.Name').fetchall()
print len(iris_l)
iris_l[:5]

150


[(u'Iris-setosa', 38),
 (u'Iris-setosa', 38),
 (u'Iris-setosa', 38),
 (u'Iris-setosa', 38),
 (u'Iris-setosa', 38)]

In [293]:
d = pd.merge(iris_df, iris_cc_df, how='inner', left_on='Species', right_on='Name')
d.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Name,DipChromCount
0,1,5.1,3.5,1.4,0.2,Iris-setosa,Iris-setosa,38
1,2,4.9,3.0,1.4,0.2,Iris-setosa,Iris-setosa,38
2,3,4.7,3.2,1.3,0.2,Iris-setosa,Iris-setosa,38
3,4,4.6,3.1,1.5,0.2,Iris-setosa,Iris-setosa,38
4,5,5.0,3.6,1.4,0.2,Iris-setosa,Iris-setosa,38


In [296]:
d[['Species', 'DipChromCount']].head(5)

,Species,DipChromCount
0,Iris-setosa,38
1,Iris-setosa,38
2,Iris-setosa,38
3,Iris-setosa,38
4,Iris-setosa,38


### Update a table

In [307]:
cursor.execute('UPDATE ChromCount SET DipChromCount=38 WHERE Name="Iris-setosa"')

In [308]:
cursor.execute('SELECT * FROM ChromCount').fetchall()[:5]

[(u'Iris-setosa', 38), (u'Iris-versicolor', 108), (u'Iris-virginica', 70)]

In [311]:
iris_cc_df.loc[iris_cc_df.Name=='Iris-setosa', 'DipChromCount'] = 38

In [312]:
iris_cc_df

,Name,DipChromCount
0,Iris-setosa,38
1,Iris-versicolor,108
2,Iris-virginica,70


### Insert a new record or row

In [318]:
cursor.execute('INSERT INTO ChromCount (Name) '\
               'VALUES ("Iris-test")')

In [319]:
cursor.execute('SELECT * FROM ChromCount').fetchall()

[(u'Iris-setosa', 38),
 (u'Iris-versicolor', 108),
 (u'Iris-virginica', 70),
 (u'Iris-test', None)]

In [322]:
cursor.execute('INSERT INTO ChromCount (DipChromCount) '\
               'VALUES (10)')

IntegrityError: NOT NULL constraint failed: ChromCount.Name

In [323]:
cursor.execute('INSERT INTO ChromCount (Name, DipChromCount) '\
               'VALUES ("Iris-test", 10)')

IntegrityError: UNIQUE constraint failed: ChromCount.Name

In [324]:
cursor.execute('INSERT INTO ChromCount (Name, DipChromCount) '\
               'VALUES ("Iris-test2", 10)')

In [325]:
cursor.execute('SELECT * FROM ChromCount').fetchall()

[(u'Iris-setosa', 38),
 (u'Iris-versicolor', 108),
 (u'Iris-virginica', 70),
 (u'Iris-test', None),
 (u'Iris-test2', 10)]

In [352]:
new_df = pd.DataFrame({'Name':['Iris-test', 'Iris-test2'],
                      'DipChromCount':[12, 10]})
iris_cc_df = iris_cc_df.append(new_df, ignore_index=True)

In [353]:
iris_cc_df

,DipChromCount,Name
0,38,Iris-setosa
1,108,Iris-versicolor
2,70,Iris-virginica
3,12,Iris-test
4,10,Iris-test2


### Delete a record/row

In [328]:
cursor.execute('DELETE FROM ChromCount '\
               'WHERE Name LIKE "Iris-test%"')

In [418]:
cursor.execute('SELECT * FROM ChromCount').fetchall()

[(u'Iris-setosa', 38), (u'Iris-versicolor', 108), (u'Iris-virginica', 70)]

In [367]:
iris_cc_df.drop(iris_cc_df[iris_cc_df.Name.str.match('Iris-test')].index)

,DipChromCount,Name
0,38,Iris-setosa
1,108,Iris-versicolor
2,70,Iris-virginica


In [369]:
iris_cc_df = iris_cc_df[~iris_cc_df.Name.str.match('Iris-test')]

In [370]:
iris_cc_df

,DipChromCount,Name
0,38,Iris-setosa
1,108,Iris-versicolor
2,70,Iris-virginica


### Insert a new attribute or column

In [407]:
# get all columns
cursor.execute('select sql FROM sqlite_master '\
               'WHERE tbl_name="ChromCount"').fetchall()

[(u'CREATE TABLE "ChromCount" (\n\t`Name`\tTEXT NOT NULL,\n\t`DipChromCount`\tNUMERIC, Test,\n\tPRIMARY KEY(`Name`)\n)',),
 (None,)]

In [411]:
cursor.execute('ALTER TABLE ChromCount '\
               'ADD Test INTEGER')

In [419]:
cursor.execute('SELECT * FROM ChromCount').fetchall()

[(u'Iris-setosa', 38), (u'Iris-versicolor', 108), (u'Iris-virginica', 70)]

In [387]:
iris_cc_df.insert(len(iris_cc_df.columns), 'Test', [10,20,30])

In [388]:
iris_cc_df

,DipChromCount,Name,Test
0,38,Iris-setosa,10
1,108,Iris-versicolor,20
2,70,Iris-virginica,30


### Delete a column

In [414]:
# NOT SUPPORTED BY SQLITE
cursor.execute('ALTER TABLE "ChromCount" DROP COLUMN Test')

OperationalError: near "DROP": syntax error

In [389]:
iris_cc_df = iris_cc_df.drop('Test', axis=1)

In [390]:
iris_cc_df

,DipChromCount,Name
0,38,Iris-setosa
1,108,Iris-versicolor
2,70,Iris-virginica
